In [1]:
import data_preproc as dp
import correlation_cognitive_methylation as ccm
import division_into_classes as dic

files = {'cpg_values': 'D:/unn/GSE52588/GSE52588_beta_fn.npz',
         'cpg_names': 'D:/unn/GSE52588/GSE52588_beta_fn.npz', 
         'data_samples': 'D:/unn/GSE52588/GSE52588_samples.txt',
         'cognitive_frame': 'D:/unn/GSE52588/DOWN_FENOTIPO_No4,8,12_PerCorrelazioni.tsv',
         'cpgs_annotations': 'D:/unn/GSE52588/cpgs_annotations.txt'} 
directory_research = 'D:/unn/down_syndrome_epigenetic'
cognitive_indicator = 'go-nogo'
alpha = 0.001

methylation_frame, cognitive_frame, indicator_folder = dp.data_preproc(files, directory_research, cognitive_indicator)
correlation_frame = ccm.correlation_cognitive_methylation(alpha, files, indicator_folder, cognitive_indicator, 
                                                      methylation_frame, cognitive_frame)

classes_all_members, classes_unique_members = dic.division_into_classes (cognitive_indicator, cognitive_frame)
print('Division into classes \nall values:', *classes_all_members, '\nunique values:', *classes_unique_members)
division_method = int(input('\nChoose how to divide into classes:\n1. all values\n2. unique values\n3. manual division\n'))
if division_method == 3:
    print(sorted(list(cognitive_frame[cognitive_indicator])))
    lim1, lim2 = [int(i) for i in input('Enter boundary values: ').split(' ')]
    indicator_classes = dic.division_into_classes(cognitive_indicator, cognitive_frame, division_method, lim1, lim2)
else:
    indicator_classes = dic.division_into_classes(cognitive_indicator, cognitive_frame, division_method)

100%|█████████████████████████████████████████████████████| 422802/422802 [06:18<00:00, 1117.64it/s]


In [ ]:
def quality_metrics (y_test, y_pred):
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average = 'macro')
    return acc, f1

def Grid_Search_CV (model, params, X_train, X_test, 
                    y_train, y_test, acc_lst, f1_lst, 
                    best_params_lst, cv_err_lst):
    grid_cv = GridSearchCV(estimator = model, param_grid = params, cv = 5, n_jobs = -1)
    grid_cv.fit(X_train, y_train)
    best_model, best_params, cv_err = grid_cv.best_estimator_, grid_cv.best_params_, grid_cv.best_score_
    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)
    
    acc, f1 = quality_metrics (y_test, y_pred)
    acc_lst.append(acc)
    f1_lst.append(f1)
    best_params_lst.append(best_params)
    cv_err_lst.append(cv_err)
    
    return best_model, best_params_lst, cv_err_lst, acc_lst, f1_lst

def ML_classification (correlation_frame, methylation_frame, indicator_classes, cognitive_indicator, indicator_folder):
    X = methylation_frame[list(correlation_frame.index)]
    y = indicator_classes['class']
    X = (X - X.mean(axis = 0)) / X.std(axis = 0)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 111, stratify = y)
    best_params_lst, cv_err_lst = [], []
    acc_lst, f1_lst = [], []
    
    dtree_model = tree.DecisionTreeClassifier(random_state = 111)
    params = {'max_depth': range (1, 4, 1)}
    dtree_best_model, best_params_lst, cv_err_lst, acc_lst, f1_lst = Grid_Search_CV(dtree_model, params, X_train, X_test, 
                                                                                    y_train, y_test, acc_lst, f1_lst, 
                                                                                    best_params_lst, cv_err_lst)
    rf_model = RandomForestClassifier(random_state = 111)
    params = {'n_estimators': range(10, 110, 10)}
    rf_best_model, best_params_lst, cv_err_lst, acc_lst, f1_lst = Grid_Search_CV(rf_model, params, X_train, X_test, 
                                                                                 y_train, y_test, acc_lst, f1_lst, 
                                                                                 best_params_lst, cv_err_lst)
    xg_model = xgb.XGBClassifier(objective ='multi:softprob', random_state = 111)
    params = {'max_depth': range (1, 4, 1), 'n_estimators': range(10, 110, 10), 
              'learning_rate': [0.1, 0.01, 0.05]}
    xgb.set_config(verbosity = 0)
    xg_best_model, best_params_lst, cv_err_lst, acc_lst, f1_lst = Grid_Search_CV(xg_model, params, X_train, X_test, 
                                                                                 y_train, y_test, acc_lst, f1_lst, 
                                                                                 best_params_lst, cv_err_lst)
    cb_model = cb.CatBoostClassifier(loss_function = 'MultiClass', random_state = 111)
    params = {'iterations': [100, 150, 200], 'learning_rate': [0.03, 0.1], 
              'depth': [2, 3, 4], 'l2_leaf_reg': [0.2, 0.5, 1, 3]}
    cb_best_model, best_params_lst, cv_err_lst, acc_lst, f1_lst = Grid_Search_CV(cb_model, params, X_train, X_test, 
                                                                                 y_train, y_test, acc_lst, f1_lst, 
                                                                                 best_params_lst, cv_err_lst)
    
    lda_model = LDA()
    params = [{'solver': ['svd', 'lsqr', 'eigen']}]
    lda_best_model, best_params_lst, cv_err_lst, acc_lst, f1_lst = Grid_Search_CV(lda_model, params, X_train, X_test, 
                                                                                  y_train, y_test, acc_lst, f1_lst, 
                                                                                  best_params_lst, cv_err_lst)
    qda_model = QDA()
    params = [{'reg_param': [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]}]
    qda_best_model, best_params_lst, cv_err_lst, acc_lst, f1_lst = Grid_Search_CV(qda_model, params, X_train, X_test, 
                                                                                  y_train, y_test, acc_lst, f1_lst, 
                                                                                  best_params_lst, cv_err_lst)
    logist = LogisticRegression(solver = 'liblinear')
    params = [{'C': np.logspace(-3, 3, 7), 'penalty': ['l1', 'l2']}]
    logist_best_model, best_params_lst, cv_err_lst, acc_lst, f1_lst = Grid_Search_CV(logist, params, X_train, X_test, 
                                                                                     y_train, y_test, acc_lst, f1_lst, 
                                                                                     best_params_lst, cv_err_lst)
    svc_linear = svm.SVC(kernel = 'linear')
    params = [{'C': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0]}]
    best_svc_linear, best_params_lst, cv_err_lst, acc_lst, f1_lst = Grid_Search_CV(svc_linear, params, X_train, X_test, 
                                                                                   y_train, y_test, acc_lst, f1_lst, 
                                                                                   best_params_lst, cv_err_lst)
    svc_rbf = svm.SVC(kernel = 'rbf')
    params = [{'gamma': [0.0001, 0.001, 0.01, 0.1, 1.0, 2.0], 'coef0': [0.0001, 0.001, 0.01, 0.1, 0.2, 1.0],
              'C': [40, 50, 60, 70, 80, 90]}]
    best_svc_rbf, best_params_lst, cv_err_lst, acc_lst, f1_lst = Grid_Search_CV(svc_rbf, params, X_train, X_test, 
                                                                                y_train, y_test, acc_lst, f1_lst, 
                                                                                best_params_lst, cv_err_lst)
    svc_poly = svm.SVC(kernel = 'poly')
    params = [{'gamma': [0.0001, 0.001, 0.01, 0.1, 1.0, 2.0], 'coef0': [0.0001, 0.001, 0.01, 0.1, 0.2, 1.0], 
              'C': [0.001, 0.01, 0.1, 1.0, 2.0, 5.0]}]
    best_svc_poly, best_params_lst, cv_err_lst, acc_lst, f1_lst = Grid_Search_CV(svc_poly, params, X_train, X_test, 
                                                                                 y_train, y_test, acc_lst, f1_lst, 
                                                                                 best_params_lst, cv_err_lst)
    best_models = [dtree_best_model, rf_best_model, xg_best_model, cb_best_model, lda_best_model, 
                   qda_best_model, logist_best_model, best_svc_linear, best_svc_rbf, best_svc_poly]
    models_names = ['Desicion Tree', 'Random Forest', 'XGBoost', 'Catboost', 'LDA', 'QDA', 'Logistic regression', 
                    'SVC (linear kernel)', 'SVC (rbf kernel)', 'SVC (poly kernel)']
    
    CL_models = open('{0}/CL_models.txt'.format(indicator_folder), 'w')
    for i in range(0, len(models_names)):
        shap_interpretation(best_models[i], models_names[i], X, X_train, directory_folder_ML)
        text = str(models_names[i]) + '\nbest params: ' + str(best_params_lst[i]) + '\nCV error = ' + str(cv_err_lst[i]) + '\nAccuracy = ' + str(acc_lst[i]) + '\nf1 = ' + str(f1_lst[i]) + '\n\n'
        CL_models.write(text)
    CL_models.close()
    return best_params_lst, cv_err_lst, acc_lst, f1_lst

In [ ]:
X = methylation_frame[correlation_frame.index].values
y = list(indicator_classes['class'])

X = (X - X.mean(axis = 0)) / X.std(axis = 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 111, stratify = y)

model = LogisticRegression(solver = 'liblinear')
params = [{'C': np.logspace(-3, 3, 7), 'penalty': ['l1', 'l2']}]

grid_cv = GridSearchCV(estimator = model, param_grid = params, scoring = 'accuracy', cv = 5, n_jobs = -1)
grid_cv.fit(X_train, y_train)
    
best_model = grid_cv.best_estimator_
best_model.fit(X_train, y_train)
best_params = grid_cv.best_params_
cv_err = grid_cv.best_score_
y_pred = best_model.predict(X_test)

explainer = shap.KernelExplainer(best_model.predict, X)
shap_values = explainer.shap_values(X)
    
indicator_folder_SHAP = '{0}/SHAP'.format(indicator_folder)
if not os.path.isdir(indicator_folder_SHAP):
        os.mkdir(indicator_folder_SHAP)

fig = plt.gcf()
x, y = 15, 12
fig_inch = (x/2.54, y/2.54)
fig, ax = plt.subplots(figsize = fig_inch)
shap.summary_plot(shap_values, features = X, feature_names = correlation_frame.index, max_display = 50, show = False)
ax = plt.gca()
fig.savefig('{0}/beeswarm.png'.format(indicator_folder_SHAP), format = 'png', dpi = 300, bbox_inches = 'tight')
k = 0
for i in shap_values:
    fig = plt.gcf()
    shap.plots._waterfall.waterfall_legacy(explainer.expected_value, shap_values[k], feature_names = correlation_frame.index)
    fig.savefig('{0}/waterfall_legacy_{1}.png'.format(indicator_folder_SHAP, k), 
                format = 'png', dpi = 300, bbox_inches = 'tight')
    k += 1
    plt.close()